In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('../../')
os.getcwd()

'/home/linh/projects/IConNet'

In [16]:
import glob
import numpy as np
import pandas as pd
import soundfile as sf

In [4]:
data_dir = '../data/physio2016/physionet.org/files/challenge-2016/1.0.0/'

In [23]:
def load_audio(filepath):
    y, sr = sf.read(filepath)
    return y
    
def read_split(split_code):
    split = f'training-{split_code}'
    split_path = f'{data_dir}{split}/'
    df = pd.read_csv(f'{split_path}REFERENCE.csv', 
        header=None, 
        names=['filename', 'label'])
    df['audio'] = df['filename'].apply(
        lambda x: load_audio(split_path+x+'.wav'))
    classname = {1: 'abnormal', -1: 'normal'}
    df['classname'] = df['label'].apply(
        lambda x: classname[x]
    )
    label_transform = {'abnormal': 0, 'normal':1}
    df['label'] = df['classname'].apply(
        lambda x: label_transform[x])
    df['length'] = df['audio'].apply(len)
    print(df['classname'].value_counts())
    return df

In [21]:
df = read_split('a')
df

classname
abnormal    292
normal      117
Name: count, dtype: int64


,filename,label,audio,classname
0,a0001,0,"[-0.002105712890625, -0.004119873046875, -0.00...",abnormal
1,a0002,0,"[0.009063720703125, 0.022216796875, 0.02377319...",abnormal
2,a0003,0,"[-0.00384521484375, -0.00933837890625, -0.0080...",abnormal
3,a0004,0,"[0.01068115234375, 0.021453857421875, 0.018280...",abnormal
4,a0005,0,"[-0.001678466796875, -0.002960205078125, -0.00...",abnormal
...,...,...,...,...
404,a0405,1,"[-0.011016845703125, -0.0216064453125, -0.0177...",normal
405,a0406,1,"[-0.014312744140625, -0.032135009765625, -0.02...",normal
406,a0407,1,"[-0.003631591796875, -0.0062255859375, -0.0024...",normal
407,a0408,1,"[-0.008880615234375, -0.019378662109375, -0.01...",normal


In [22]:
df['length'] = df['audio'].apply(len)
df['length'].describe()

count      409.000000
mean     65174.122249
std      10577.780210
min      18530.000000
25%      61580.000000
50%      71193.000000
75%      71472.000000
max      73004.000000
Name: length, dtype: float64

In [24]:
splits = ['b','c','d','e','f']
for code in splits:
    dfa = read_split(code)
    df = pd.concat([df, dfa])
df

classname
normal      386
abnormal    104
Name: count, dtype: int64
classname
abnormal    24
normal       7
Name: count, dtype: int64
classname
abnormal    28
normal      27
Name: count, dtype: int64
classname
normal      1958
abnormal     183
Name: count, dtype: int64
classname
normal      80
abnormal    34
Name: count, dtype: int64


,filename,label,audio,classname,length
0,a0001,0,"[-0.002105712890625, -0.004119873046875, -0.00...",abnormal,71332
1,a0002,0,"[0.009063720703125, 0.022216796875, 0.02377319...",abnormal,41657
2,a0003,0,"[-0.00384521484375, -0.00933837890625, -0.0080...",abnormal,61440
3,a0004,0,"[0.01068115234375, 0.021453857421875, 0.018280...",abnormal,71193
4,a0005,0,"[-0.001678466796875, -0.002960205078125, -0.00...",abnormal,71332
...,...,...,...,...,...
109,f0110,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",normal,61344
110,f0111,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",normal,58752
111,f0112,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",normal,60864
112,f0113,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",normal,65952


In [25]:
df['label'].value_counts()

label
1    2575
0     665
Name: count, dtype: int64

In [26]:
df['length'].describe()

count      3240.000000
mean      44923.505864
std       24772.785120
min       10611.000000
25%       25798.500000
50%       41657.000000
75%       61162.000000
max      243997.000000
Name: length, dtype: float64

In [29]:
preprocess_dir = '../data/data_preprocessed/heartsound/preprocessed/'

In [30]:
np.save(f'{preprocess_dir}heartsound.audio2k.train.npy', 
        df['audio'], 
allow_pickle=True)

In [31]:
np.save(f'{preprocess_dir}heartsound.label_binary.train.npy', 
        df['label'], 
allow_pickle=True)